# Q&A Agent With Graph DB

Building a Question and Answering application over a Graph Database

# Setup 

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

## Openai
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")


# Nvidia
os.environ["NVIDIA_API_KEY"] = os.getenv("NVIDIA_API_KEY")

## langsmith tracking
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")

## Huggingface
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")

## Neo4j
os.environ["NEO4J_URI"] = os.getenv("NEO4J_URI")
os.environ["NEO4J_USERNAME"] = os.getenv("NEO4J_USERNAME")
os.environ["NEO4J_PASSWORD"] = os.getenv("NEO4J_PASSWORD")


In [7]:
from langchain_community.graphs import Neo4jGraph

graph = Neo4jGraph(
    url=os.environ["NEO4J_URI"],
    username=os.environ["NEO4J_USERNAME"],
    password=os.environ["NEO4J_PASSWORD"],
)

## Import dataset

I woul like to cite and thanks [Tomaz Bratanic](https://github.com/tomasonjo) -- Graph ML and GenAI research at Neo4j -- to provide your public [datasets](https://github.com/tomasonjo/blog-datasets).

In [12]:
## Dataset Movie reviews

movie_query = """
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' AS row

MERGE (m:Movie{movieId: toInteger(row.movieId)})
SET m.title = row.title,
    m.release = datetime(row.release),
    m.imdbRating = toFloat(row.imdbRating)

FOREACH (director IN split(row.director, '|') |
    MERGE (d:Director{name: trim(director)})
    MERGE (m)-[:DIRECTED_BY]->(d)
    MERGE (d)-[:DIRECTOR_OF]->(m)
)

FOREACH (actor IN split(row.actors, '|') |
    MERGE (a:Actor{name: trim(actor)})
    MERGE (a)-[:ACTOR_OF]->(m)
    MERGE (m)-[:HAS_ACTOR]->(a)
)

FOREACH (genre IN split(row.genres, '|') |
    MERGE (g:Genre{name: trim(genre)})
    MERGE (m)-[:HAS_GENRE]->(g)
    MERGE (g)-[:GENRE_OF]->(m)
)
"""


In [13]:
graph.query(movie_query)

Failed to read from defunct connection IPv4Address(('si-01de3140-caed.production-orch-0696.neo4j.io', 7687)) (ResolvedIPv4Address(('34.28.184.63', 7687)))


[]

#### 📊 **Gaph Relatioship Illustration of `ACTOR_OF`:**

![Relação ACTOR_OF](image/actor_of.png)


#### 📊 **Gaph Relatioship Illustration of `HAS_GENRE`:**

![Relação ACTOR_OF](image/genre.png)

In [15]:
graph.refresh_schema()
print(graph.schema)

Failed to read from defunct connection IPv4Address(('si-01de3140-caed.production-orch-0696.neo4j.io', 7687)) (ResolvedIPv4Address(('34.28.184.63', 7687)))


Node properties:
CEO {name: STRING, POB: STRING, YOB: INTEGER}
Company {name: STRING}
User {name: STRING, city: STRING, userId: INTEGER, age: INTEGER}
Post {userId: INTEGER}
Movie {movieId: INTEGER, title: STRING, imdbRating: FLOAT}
Director {name: STRING}
Actor {name: STRING}
Genre {name: STRING}
Relationship properties:

The relationships:
(:CEO)-[:CEO]->(:Company)
(:User)-[:POSTED]->(:Post)
(:User)-[:FRIEND]->(:User)
(:User)-[:LIKES]->(:User)
(:Movie)-[:DIRECTED_BY]->(:Director)
(:Movie)-[:HAS_ACTOR]->(:Actor)
(:Movie)-[:HAS_GENRE]->(:Genre)
(:Director)-[:DIRECTOR_OF]->(:Movie)
(:Actor)-[:ACTOR_OF]->(:Movie)
(:Genre)-[:GENRE_OF]->(:Movie)
